In [9]:
import os
import sys
import numpy as np
import cv2

from homography import compute_h, cor_p
from triangulation import triangulation
from pathlib import Path
from shapely.geometry import Polygon

In [10]:
num_cams = 4
cam_poses = {} # key: cami, value: pose
for i in range(num_cams):
    with open(f'./camera_poses/{i:05d}.txt', 'r') as f:
        lines = f.readlines()
        pose = []
        for line in lines:
            data = list(map(float, line.split(" ")))
            pose.append(data)
        pose = np.array(pose)
        cam_poses[f'cam{i}'] = pose.reshape(4, 4)

In [11]:
yolo_outputs = {} # key: cami, value: yolo_output

for i in range(num_cams):

    # outputs of one camera:
    yolo_output = {} # key: class, value: yolo_output
    yolo_output['chair'], yolo_output['table'], yolo_output['person'] = [], [], []

    with open(f'./runs/detect/layout/cam{i}/labels/00000.txt', 'r') as f:
        lines = f.readlines()
        for line in lines:
            data = list(map(float, line.split(" ")))

            # check object class
            if data[0] == 56 or data[0] == 11: # chair or bench
                yolo_output['chair'].append(np.array(data[1:5]))
            elif data[0] == 60: # table
                yolo_output['table'].append(np.array(data[1:5]))
            elif data[0] == 0: # person
                yolo_output['person'].append(np.array(data[1:5]))
        yolo_outputs[f'cam{i}'] = yolo_output

In [8]:
h, w = 1456, 1928

poses = [cam_poses[f'cam{i}'][:3, :] for i in range(num_cams)]

K = np.array([
        [975.813843, 0, 960.973816],
        [0, 975.475220, 729.893921],
        [0, 0, 1]
    ])

for idx in range(len(cor_p)):
    X = [cor_p[i][idx][0] for i in range(num_cams)]
    Y = [cor_p[i][idx][1] for i in range(num_cams)]
    print(X, Y)
    triangulated = triangulation(poses, X, Y)
    print(triangulated)
    for i in range(num_cams):
        reversed = K @ poses[i] @ np.array(triangulated.tolist() + [1])
        reversed = [reversed[0]/reversed[-1], reversed[1]/reversed[-1]]
        print(f"Reversed{i}: ", reversed)

[609, 1223, 1613, 1673] [1025, 1082, 893, 698]
calib:
[[9.75813843e+02 2.84402057e-16 9.60973816e+02 1.06689526e-13]
 [8.24046376e-15 9.75475220e+02 7.29893921e+02 8.10345037e-14]
 [3.39732582e-17 1.17137382e-17 1.00000000e+00 1.11022302e-16]]
calib:
[[ 5.84453355e+02  5.40118102e+00  1.23857516e+03 -1.99600365e+03]
 [ 2.62476672e-01  1.08122843e+03  5.61464214e+02 -1.94220671e+02]
 [-3.21644952e-01  1.93865167e-01  9.26801393e-01 -1.41759157e-01]]
calib:
[[ 1.01487110e+01 -7.04799419e+01  1.36770352e+03 -2.71137296e+03]
 [-3.36866115e+01  1.07697409e+03  5.68585138e+02 -9.98959371e+01]
 [-6.34520688e-01  2.76914668e-01  7.21596676e-01  9.39751345e-01]]
calib:
[[-7.38828945e+02 -2.16202645e+02  1.13272748e+03 -1.02382614e+03]
 [-1.78462610e+02  9.87229618e+02  6.91249462e+02  6.10443337e+01]
 [-8.75331302e-01  3.41241052e-01  3.42563361e-01  3.03963220e+00]]
[-2.6779146  -0.62920965  4.26402548]
Reversed0:  [348.13832794045675, 585.9504986388437]
Reversed1:  [377.35953404468864, 333.85